# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [26]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\franc\Desktop\Big Data Engineering\project_2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if row[0] == '':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [28]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

3626


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [29]:
# DB connection
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [30]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music_plays
        WITH REPLICATION =
        {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [31]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_plays')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



#### Query 1

In [32]:
drop_table_query1 = "DROP TABLE IF EXISTS songs_session"

# query 1 requires sessionId and itemsInSession in the WHERE clause.
# added a composed primary key using sessionId (partition key) and itemsInSession (clustering column)
create_table_query1 = """
    CREATE TABLE IF NOT EXISTS songs_session (
    sessionId INT,
    itemInSession INT,
    artistName TEXT,
    songTitle TEXT,
    songLength FLOAT,
    PRIMARY KEY (sessionId, itemInSession))
"""

try:
    session.execute(drop_table_query1)
    session.execute(create_table_query1)
except Exception as e:
    print(e)

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT data into songs_session
        query = "INSERT INTO songs_session (sessionId, itemInSession, artistName, songTitle, songLength) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [34]:
## check on data inserted in songs_session

select_query_1 = """
    SELECT artistName, songTitle, songLength
    FROM songs_session
    WHERE sessionId=338 AND itemInSession=4"""

try:
    rows = session.execute(select_query_1)
    for row in rows:
        print (row.artistname, row.songtitle, row.songlength)
except Exception as e:
    print(e)

#double check on the import file using pandas
df = pd.read_csv(file)
df.loc[(df['sessionId'] == 338) & (df['itemInSession'] == 4), ['artist', 'song', 'length']]

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


,artist,song,length
964,Faithless,Music Matters (Mark Knight Dub),495.3073


#### Query 2

In [35]:
drop_table_query2 = "DROP TABLE IF EXISTS songs_user"

# query 2 requires userId and sessionId in the WHERE clause.
# Added a composed primary key using userId (partition key) and sessionId / itemsInSession as clustering columns.
# We uniquely identify every songs played by each user in different sessions, however we accept only one song for
# each user session (think about multiple songs listed by the user in the same session).
create_table_query2 = """
    CREATE TABLE IF NOT EXISTS songs_user (
    userId INT,
    sessionId INT,
    itemInSession INT,
    artistName TEXT,
    songTitle TEXT,
    userFirstName TEXT,
    userLastName TEXT,
    PRIMARY KEY (userId, sessionId, itemInSession))
"""

try:
    session.execute(drop_table_query2)
    session.execute(create_table_query2)
except Exception as e:
    print(e)

In [36]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT data into songs_user
        query = "INSERT INTO songs_user (userId, sessionId, itemInSession, artistName, songTitle, userFirstName, userLastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
# check on data inserted in songs_user

select_query_2 = """
    SELECT artistname, songtitle, userfirstname, userlastname
    FROM songs_user
    WHERE userid=10 AND sessionid=182"""

try:
    rows = session.execute(select_query_2)
    for row in rows:
        print (row.artistname, row.songtitle, row.userfirstname, row.userlastname)
except Exception as e:
    print(e)

#double check on the import file using pandas
df = pd.read_csv(file)
df.sort_values(by='itemInSession').loc[(df['userId'] == 10) & (df['sessionId'] == 182), ['artist', 'song', 'firstName', 'lastName']]

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


,artist,song,firstName,lastName
151,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
152,Three Drives,Greece 2000,Sylvie,Cruz
153,Sebastien Tellier,Kilometer,Sylvie,Cruz
154,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3

In [38]:
drop_table_query3 = "DROP TABLE IF EXISTS songs_listened"

# query 3 requires songTitle in the WHERE clause.
# Added a composed primary key using songTitle (partition key) and userId / sessionId as clustering columns.
create_table_query3 = """
    CREATE TABLE IF NOT EXISTS songs_listened (
    userId INT,
    sessionId INT,
    songTitle TEXT,
    userFirstName TEXT,
    userLastName TEXT,
    PRIMARY KEY (songTitle, userId, sessionId))
"""

try:
    session.execute(drop_table_query3)
    session.execute(create_table_query3)
except Exception as e:
    print(e)

In [39]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT data into songs_listened
        query = "INSERT INTO songs_listened (userId, sessionId, songTitle, userFirstName, userLastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
# check on data insert in songs_listened

select_query_3 = """
    SELECT userfirstname, userlastname
    FROM songs_listened
    WHERE songTitle='All Hands Against His Own' """

try:
    rows = session.execute(select_query_3)
    for row in rows:
        print (row.userfirstname, row.userlastname)
except Exception as e:
    print(e)

#double check on the import file using pandas
df = pd.read_csv(file)
df.loc[(df['song'] == 'All Hands Against His Own'), ['firstName', 'lastName']]


Jacqueline Lynch
Tegan Levine
Sara Johnson


,firstName,lastName
219,Sara,Johnson
2442,Jacqueline,Lynch
2646,Tegan,Levine


### Drop the tables before closing out the sessions

In [41]:
try:
    session.execute(drop_table_query1)
    session.execute(drop_table_query2)
    session.execute(drop_table_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()
